In [ ]:
import get_input

In [3]:
print(ord('0'))
print(ord('9'))
print(ord('-'))

48
57
45


In [ ]:
string = get_input(2015, 12)

In [ ]:
s = ""

for c in string:
    if ord(c) == 48 or ord(c) == 57 or ord(c) == 45:
        s += c